# HW note

## Calib
### fix size & fix focal length ➡️ .npz

In [4]:
import cv2
import CalibrationHelpers as calib
import glob
import time
directory = ''
imagesp = glob.glob(directory+'/*.png')
imagesj = glob.glob(directory+'/*.jpeg')
res = 400
for fname in imagesp + imagesj:
    img = cv2.imread(fname)
    #  downscale your image 
    img = cv2.resize(img,(RES,RES))

## Step 4 Compute relative pose between cameras

In [ ]:
# import numpy as np
import Mobile3DRecon
images = glob.glob('Mobile_Ref_data'+'/*.jpeg')
savR = np.zeros((len(images), 3, 3))
savT = np.zeros((len(images), 3))
imgNum = 0
intrinsics, distortion, new_intrinsics, roi = \
        calib.LoadCalibrationData('mobile_calib_data')
RES = 400
for fname in images:
    # read the image
    print(fname)
    cap = cv2.imread(fname)
    current_frame = cv2.resize(cap,(RES,RES))
    ret, R, T = Mobile3DRecon.ComputePoseFromHomography(new_intrinsics,referencePoints,
                                          imagePoints)
    savR[imgNum] = R
    savT[imgNum] = T
    imgNum += 1
    
# Compute relative pose between cameras
relR = np.zeros((len(images), 3, 3))
relT = np.zeros((len(images), 3))
fromimg = 0
for i in range(imgNum - 1):
    relR[i + 1] = np.matmul(savR[i + 1], savR[fromimg])
    relT[i + 1] = savT[i + 1] - np.matmul(np.matmul(savR[i + 1], savR[fromimg].T), savT[fromimg])

print("relR", relR[1:])
print("relT", relT[1:])

In [ ]:
import scipy.io as spio
spio.savemat("relR.mat", relR)


## Step 5 Compute feature matches between images

In [ ]:
images = glob.glob('Mobile_Ref_data'+'/*.jpeg')
# Load the reference image that we will try to detect in the webcam
reference = cv2.imread(images[0],0)

## Step 6 Use epipolar constraints to remove bad feature matches
